In [1]:
import pandas as pd
import warnings; warnings.simplefilter('ignore')

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [16]:
df_trans = pd.read_csv('./data/trans.csv', index_col=0)

In [17]:
df_trans.columns

Index(['fullVisitorId', 'is_transaction', 'transactionRevenue', 'bounces',
       'hits', 'pageviews', 'visitNumber', 'is_source_googleplex',
       'is_source_direct', 'is_source_google', 'is_medium_referral',
       'is_medium_none', 'is_medium_organic', 'is_bayarea', 'Africa',
       'Americas', 'Asia', 'Europe', 'Oceania', 'next_session_1',
       'last_session_1', 'q1', 'q2', 'q3', 'q4', 'weekday', 'h_0_6', 'h_7_12',
       'h_13_17', 'h_18_23', 'browser_chrome', 'is_mobile', 'system_google'],
      dtype='object')

## split

In [28]:
X = df_trans.drop(['is_transaction', 'fullVisitorId','transactionRevenue'], axis=1)
y = df_trans['transactionRevenue']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

## Stacking

In [29]:
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np

class StackedRegressor(BaseEstimator, ClassifierMixin):  
    def __init__(self, classifier, regressor):
        self.classifier = classifier
        self.regressor = regressor
#         args, _, _, values = inspect.getargvalues(inspect.currentframe())
#         values.pop("self")
#         for arg, val in values.items():
#             for param, value in val.get_params().items():
#                 arg_final = arg + '_' + ''
#                 setattr(self, )
        
    def fit(self, X, y):
        class_labels = pd.Series(np.where(y>0,1,0))
        self.classifier.fit(X,class_labels)
        
        X = X.reset_index(drop=True)
        class_labels = class_labels.reset_index(drop=True)
        y = y.reset_index(drop=True)    
        X_reg = X[class_labels==1]
        y = y[class_labels==1]
        self.regressor.fit(X_reg,y)
        
    def predict(self, X):
        class_predict = self.classifier.predict(X)
        regressor_predict = self.regressor.predict(X)
        np.multiply(class_predict,regressor_predict)
        return np.multiply(class_predict,regressor_predict)
    
    def score(self, X, y):
        return -np.sqrt(np.mean((y - self.predict(X))**2))

## Grid Search

In [30]:
rfc = []
for n in [30, 50]:
    for min_l in [3, 5]:
        rfc.append(RandomForestClassifier(random_state=42, n_estimators = n, min_samples_leaf = min_l))

In [31]:
logic = []
for penalty in [ 'l1', 'l2']:
    for C in [0.5, 0.7, 1]:
        logic.append(LogisticRegression(penalty=penalty, C=C, random_state=42))

In [32]:
lr = []
for normalize in [True, False]:
    lr.append(LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=None))

In [33]:
rfr = []
for n in [30, 50]:
    for min_l in [3, 5]:
        rfr.append(RandomForestRegressor(random_state=42, n_estimators = n, min_samples_leaf = min_l))

In [34]:
parameters = {'classifier': rfc + logic, 
              'regressor': lr + rfr}

In [164]:
gs = GridSearchCV(cv=5, estimator=StackedRegressor(RandomForestClassifier(random_state=42),
                                                   LinearRegression()),param_grid=parameters)

In [ ]:
gs.fit(X=X_train, y=regressor_train)

In [118]:
gs.best_estimator_

StackedRegressor(classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
         regressor=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False))

In [132]:
gs.get_params()['param_grid']['classifier'][0].get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

- LogisticRegression + RandomForestRegression is best combination by default

In [119]:
gs.best_score_

-2.0282318957517735